In [5]:
import tabint
from tabint import bi_model
from tabint.bi_model import *

In [6]:
link = 'localhost:59228'

In [10]:
dax_string = '''

// DAX Query
DEFINE VAR __DS0FilterTable = 
  FILTER(
    KEEPFILTERS(VALUES('Date'[Date])),
    'Date'[Date] >= DATE(2020, 6, 29)
  )

EVALUATE
  TOPN(
    502,
    SUMMARIZECOLUMNS(
      ROLLUPADDISSUBTOTAL(ROLLUPGROUP('booking'[booking_type], 'booking'[status]), "IsGrandTotalRowTotal"),
      __DS0FilterTable,
      "Number_of_booking", 'booking measure'[Number of booking]
    ),
    [IsGrandTotalRowTotal],
    0,
    'booking'[booking_type],
    1,
    'booking'[status],
    1
  )

ORDER BY
  [IsGrandTotalRowTotal] DESC, 'booking'[booking_type], 'booking'[status]


'''

In [8]:
model = powerbi.from_data_source(link)

In [25]:
df = model.get_DAX(dax_string)

In [26]:
df

,booking[booking_type],booking[status],[IsGrandTotalRowTotal],[Number_of_booking]
0,nan,nan,True,8790
1,instant_book,Cancel by Guest,False,258
2,instant_book,Cancel by host,False,65
3,instant_book,Completed,False,1905
4,instant_book,Incomplete,False,2656
5,instant_book,Incomplete by guest,False,8
6,instant_book,Pending,False,33
7,request_to_book,Accepted,False,10
8,request_to_book,Cancel by Guest,False,152
9,request_to_book,Cancel by host,False,13


In [27]:
df = df[df['booking[booking_type]'] != "nan"]

In [28]:
df_complete = df[df['booking[status]'] == "Completed"]

In [29]:
df_complete

,booking[booking_type],booking[status],[IsGrandTotalRowTotal],[Number_of_booking]
3,instant_book,Completed,False,1905
10,request_to_book,Completed,False,977


In [33]:
df_sum = df.groupby(by = ['booking[booking_type]'], as_index=False)['[Number_of_booking]'].sum()

In [34]:
df_sum

,booking[booking_type],[Number_of_booking]
0,instant_book,4925
1,request_to_book,3865


In [36]:
from scipy.stats import binom_test

In [37]:
binom_test(1905, 4925, 977/3865, alternative='greater')

5.2641027900340074e-95